In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
import plotly.express as px
import plotly.graph_objects as go
import networkx as nx
import itertools
import random

from utils import *

In [2]:
df = load_process_data('features.csv')
df.head()

,song,bpm,energy,key,chroma_0,chroma_1,chroma_2,chroma_3,chroma_4,chroma_5,...,mfcc_11,mfcc_12,mfcc_13,mfcc_14,mfcc_15,mfcc_16,mfcc_17,mfcc_18,mfcc_19,spectral_centroids
0,1luu - Oxygenium [T4F].mp3,158,0.287214,Fm,-0.213677,0.537364,-1.027106,-0.098883,-0.685025,-1.133333,...,0.730683,1.011450,0.246780,-0.606063,-0.032803,-0.372813,0.092517,0.093309,-0.425784,1.174197
1,A$AP ROCKY - Praise the Lord (DURDENHAUER Edit...,148,0.518222,Am,1.437601,0.013851,-0.247254,1.087473,-0.268538,-0.144419,...,-0.085135,1.567214,0.919243,1.051717,0.642097,0.283935,0.416576,0.379108,-0.688901,-1.533422
2,Alignment - Future Dancefloor [VNR043 A2].mp3,142,-0.142389,Fm,-0.990388,-0.016995,0.213866,-0.447798,0.658377,0.435928,...,0.356855,1.331722,1.354641,1.096235,1.023339,0.381407,0.537108,0.113898,0.553966,0.807301
3,"Alizée - Moi, Lolita (Prauze Edit Enfan Party)...",150,-1.044338,G#m,-0.295786,1.038825,1.358471,1.096532,1.402330,1.962330,...,0.971788,0.248500,0.495263,0.850674,-0.026536,-0.557985,-1.036313,-0.536170,-0.002231,0.918917
4,Amadou & Mariam - Sabali (RAUMM Edit).mp3,148,-0.215854,F#,-0.589446,0.021460,-1.165045,1.564127,0.341731,-0.602349,...,-0.805673,-0.124586,1.052681,0.870500,-0.283053,0.629791,1.577096,0.492321,0.177146,0.238403


In [3]:
X = df.drop(['song', 'bpm', 'key'], axis=1).values

In [4]:
# PCA to plot the data in 2D (using plotly)
pca = PCA(n_components=2)
X_pca = pca.fit_transform(X)

# Keep a column with color that change with bpm increasing (bpm being the first column of X)
X_pca = np.c_[X_pca, df['bpm'].values]

# Plot the data in 2D
fig = px.scatter(x=X_pca[:, 0], y=X_pca[:, 1], color=X_pca[:, 2])
fig.show()


In [5]:
# 'Pier - Angèle Saiyan (Techno Edit).mp3" bpm is too low, we remove it
df = df[df.song != 'Pier - Angèle Saiyan (Techno Edit).mp3']

# Assuming df is your DataFrame with songs and their BPMs
G = create_graph(df)
G = create_graph_key_constraint(df)

# Normalize BPM for color mapping
bpm_normalized = (df.set_index('song')['bpm'] - df['bpm'].min()) / (df['bpm'].max() - df['bpm'].min())

# Calculate node positions
pos = nx.spring_layout(G)

# Node trace
node_trace = go.Scatter(
    x=[pos[node][0] for node in G.nodes()],
    y=[pos[node][1] for node in G.nodes()],
    mode='markers',
    marker=dict(
        size=10,
        color=[bpm_normalized[node] for node in G.nodes()],
        colorscale='Viridis',
        colorbar=dict(title='BPM'),
        line_width=2,
        opacity=0.8  # Reduced opacity
    ),
    text=[node for node in G.nodes()],  # Hover text
    hoverinfo='text'
)

# Edge trace
edge_trace = go.Scatter(
    x=[],
    y=[],
    line=dict(width=0.5, color='#888'),
    hoverinfo='none',
    mode='lines')

for edge in G.edges():
    x0, y0 = pos[edge[0]]
    x1, y1 = pos[edge[1]]
    edge_trace['x'] += (x0, x1, None)
    edge_trace['y'] += (y0, y1, None)

# Create the figure
fig = go.Figure(data=[edge_trace, node_trace],
                layout=go.Layout(
                    title='Song Graph',
                    titlefont_size=16,
                    showlegend=False,
                    hovermode='closest',
                    margin=dict(b=20, l=5, r=5, t=40),
                    xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
                    yaxis=dict(showgrid=False, zeroline=False, showticklabels=False))
                )

fig.show()

In [ ]:
def find_shortest_hamiltonian_path(G):
    # Store the shortest path and its length
    shortest_path = None
    shortest_path_length = float('inf')

    # Iterate over all permutations of nodes to check each possible path
    for path in itertools.permutations(G.nodes()):
        current_length = 0
        valid_path = True

        # Calculate the length of the current path
        for i in range(len(path) - 1):
            if G.has_edge(path[i], path[i + 1]):
                current_length += G[path[i]][path[i + 1]]['weight']
            else:
                valid_path = False
                break

        # Update the shortest path if a shorter one is found
        if valid_path and current_length < shortest_path_length:
            shortest_path_length = current_length
            shortest_path = path

        print(f"Checked path: {path}, Length: {current_length}")

    return shortest_path, shortest_path_length


path, length = find_shortest_hamiltonian_path(G)
print(f"Shortest Hamiltonian path: {path}, Length: {length}")